In [ ]:
# pip install MetaTrader5

# Bot

In [ ]:
#Initialization 

import MetaTrader5 as mt

mt.initialize()

#Adjust the following parameters to your account
login = 
password = ""
server = ""

mt.login(login,password,server)

In [ ]:
#Actual Bot

#Imports
import talib
import numpy as np
import pandas as pd
import MetaTrader5 as mt
from datetime import datetime
import time

#General Information and details about order
ticker     = 'BTCUSD'
qty        = 0.01
buy_order  = mt.ORDER_TYPE_BUY
sell_order = mt.ORDER_TYPE_SELL
buy_price  = mt.symbol_info_tick(ticker).ask
sell_price = mt.symbol_info_tick(ticker).bid
sl_pct     = 0.05
tp_pct     = 0.1
buy_sl     = buy_price * (1-sl_pct)
buy_tp     = buy_price * (1+tp_pct)
sell_sl    = sell_price * (1+sl_pct)
sell_tp    = sell_price * (1-tp_pct)
interval   = mt.TIMEFRAME_H2
from_date  = datetime.now()

#Function to create Order
def create_order(ticker,qty,order_type,price,sl,tp):
    request={
        "action"       : mt.TRADE_ACTION_DEAL,
        "symbol"       : ticker,
        "volume"       : qty,
        "type"         : order_type,
        "price"        : price,
        "sl"           : sl,
        "tp"           : tp,
        "magic"        : 123456,
        "comment"      : "Python Open Position",
        "type_time"    : mt.ORDER_TIME_GTC,
        "type_filling" : mt.ORDER_FILLING_IOC

    }

    #Send a trading request
    order = mt.order_send(request)
    return order

#Function to Close Order
def close_order(ticker,qty,order_type,price):
    request={
        "action"       : mt.TRADE_ACTION_DEAL,
        "symbol"       : ticker,
        "volume"       : qty,
        "type"         : order_type,
        "position"     : mt.positions_get()[0]._asdict()['ticket'],
        "price"        : price,
        "comment"      : "Python Close Position",
        "type_time"    : mt.ORDER_TIME_GTC,
        "type_filling" : mt.ORDER_FILLING_IOC

    }

    #Send a trading request
    order = mt.order_send(request)
    return order

#Set current orders status to Closed
already_buy  = False
already_sell = False
no_positions = 0
    
#Range should be adjusted
for i in range(100000):
    ohlc = pd.DataFrame(mt.copy_rates_range(ticker,interval,datetime(2024,3,1),from_date))
    ohlc['time']=pd.to_datetime(ohlc['time'],unit='s')

    # Calculate indicators
    ohlc['EMA200'] = talib.EMA(ohlc['close'], timeperiod=200)
    ohlc['EMA50'] = talib.EMA(ohlc['close'], timeperiod=50)
    ohlc['RSI'] = talib.RSI(ohlc['close'], timeperiod=14)

    current_RSI = list(ohlc[-1:]['RSI'])[0]
    long_cond        = current_RSI > 69
    close_long_cond  = current_RSI < 60
    short_cond       = current_RSI < 38
    close_short_cond = current_RSI > 42
    
    #no_positions = len(mt.positions_get()) == 0

    #The Strategy itself
    if no_positions == 0:
        if long_cond:
            create_order(ticker,qty,buy_order,buy_price,buy_sl,buy_tp)
            already_buy  = 1
            no_positions = 1
            print("BUY Order Placed")
        elif short_cond:
            create_order(ticker,qty,sell_order,sell_price,sell_sl,sell_tp)
            already_sell = 1
            no_positions = 1
            print("SELL Order Place")
    
    elif already_buy == 1 and close_long_cond == 1:
        close_order(ticker,qty,sell_order,sell_price)
        no_positions = 0
        already_buy  = 0
        print("Close BUY Order")
    elif already_sell == 1 and close_short_cond == 1:
        close_order(ticker,qty,buy_order,buy_price)
        no_positions = 0
        already_sell = 0
        print("Close SELL Order")
            
    time.sleep(60)

# Backtesting

In [ ]:
#Imports
import talib
import numpy as np
import pandas as pd
import MetaTrader5 as mt
from datetime import datetime
import time


ticker     = 'BTCUSD'
qty        = 0.2
interval   = mt.TIMEFRAME_H2
from_date  = datetime.now()

df = pd.DataFrame(mt.copy_rates_range(ticker,interval,datetime(2024,3,1),from_date))
df['time']=pd.to_datetime(df['time'],unit='s')

# Calculate indicators
df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
df['RSI'] = talib.RSI(df['close'], timeperiod=14)

# Initialize variables for backtesting
position = None
initial_balance = 10000  # Starting with $10,000
balance = initial_balance 
holdings = 0
entry_price = 0
trade_log = []
balance_over_time = []


# Backtesting loop
for i in range(1, len(df)):
    current_price = df['close'].iloc[i]
    rsi = df['RSI'].iloc[i]
    timestamp = df['time'].iloc[i]

    # Record balance over time
    balance_over_time.append({'timestamp': timestamp, 'balance': balance})

    # Check if no position is currently held
    if position is None:
#         Long position entry condition
        if rsi > 70:
            position = 'LONG'
            entry_price = current_price
            holdings = qty
            balance -= entry_price * qty
            trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': position})

        # Short position entry condition
        elif rsi < 30:
            position = 'SHORT'
            entry_price = current_price
            holdings = qty
            balance += entry_price * qty
            trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': position})

    # If in a LONG position, check for exit condition
    elif position == 'LONG' and rsi < 70:
        balance += current_price * holdings
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
        position = None

    # If in a SHORT position, check for exit condition
    elif position == 'SHORT' and rsi > 30:
        balance -= current_price * holdings
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
        position = None

# Final balance calculation (accounting for any remaining positions)
if position == 'LONG':
    balance += current_price * holdings
if position == 'SHORT':
    balance -= current_price * holdings

# Output results
print(f"Initial Balance: ${initial_balance}")
print(f"Final Balance: ${balance}")
print(f"Net Profit/Loss: ${balance - initial_balance}")

# Convert trade log to DataFrame for analysis
trade_log_df = pd.DataFrame(trade_log)

# Output the trade log
print(trade_log_df)

# Save trade log to CSV
trade_log_df.to_csv('trade_log.csv', index=False)

# Convert balance over time to DataFrame
balance_over_time_df = pd.DataFrame(balance_over_time)